# GPS Tester

- Modified version of [this notebook](./example/gps_serial.ipynb)

## Data Collection from GPS Module
---

#### Data Acquisition (Acquire and save GPS NMEA Data)

##### Finding your device in `/dev`
- `ls /dev | grep -e usb -i`
  - Lists all devices with `usb` (case insensitive) in their name
- `sudo dmesg`
  - Lists events from the kernel (incl. USB events)
  - Try running the command, unplugging the device, running it again and seeing the most recent log
  - Can try plugging it back in to get the name of the `/dev` folder that represents the device

##### Ensuring you have necessary permissions to communicate with the serial device
- You will likely need to run the following script as root to get it working
- Or better yet, you can run `sudo adduser <your_username> dialout`
  - Adds your user to the group for the USB TTY devices
  - Allows access
  - Restart computer after doing so (logging out and back in doesn't work)

In [ ]:
import serial
import datetime
import time
import os
import getpass

# Globals for data acquisition
# Port the GPS module is on
gps_port = "/dev/ttyUSB0"
sentences_per_file = 700

# User check
print("Running under user: " + getpass.getuser())
print("Make sure correct IO permissions are enabled for this user")

# ser = serial.Serial("/dev/ttyUSB0", baudrate=9600)
ser = serial.Serial(port=gps_port, baudrate=9600)
ser.flushInput()
ser.flushOutput()
idx = 0

nmea_data = b""

# skip first line, since it could be incomplete
ser.readline()

while True:
    idx += 1
    nmea_sentence = ser.readline()
    nmea_data += nmea_sentence

    if idx % 100 == 0:
        print(f"idx: {idx}/{sentences_per_file}")

    # save to file after <number> sentences added
    if idx % sentences_per_file == 0:
        print("Exporting")
        filename = datetime.datetime.utcnow().strftime("data/gps_data_%Y%m%d-%H%M%S.nmea")
        f = open(filename, "ab")
        f.write(nmea_data)
        f.close()
        print(f"Exported as ./{filename}")

        nmea_data = b""

#### Listen for `$GPGSV` Messages

In [ ]:
import serial
import re

ser = serial.Serial(port=gps_port, baudrate=9600)
ser.flushInput()
ser.flushOutput()

# skip first line, since it could be incomplete
ser.readline()

while True:
    nmea_sentence = ser.readline()
    nmea_sentence_dec = nmea_sentence.decode("utf-8")
    # print(nmea_sentence_dec)

    if re.findall(pattern="[$]GPGSV", string=nmea_sentence_dec):
        print(nmea_sentence_dec, end="")

## Data Analysis
---
### Choose Most Recent File For Analysis

In [ ]:
# Globals for data analysis

# File to open for analysis
# Set to a relative filepath to specify a file
gps_data_file = None
# --------------------------------------------------------------------------------------------------


# Credit to:
# https://fabianlee.org/2021/11/11/python-find-the-most-recently-modified-file-matching-a-pattern/

import datetime
import os
import glob

# Get most recent .nmea file in GPS data folder and set that as the file we'll use
# Provided no file was given in cell above
if not gps_data_file:
    # Get list of files that matches pattern
    pattern = r"./data/gps_data_[0-9]*-[0-9]*[.]nmea"
    print(pattern)
    files = list(filter(os.path.isfile, glob.glob(pattern)))

    # Sort by modified time
    files.sort(key=lambda x: os.path.getmtime(x))

    # Get last item in list
    gps_data_file = files[-1]

    # Print info
    now_str = datetime.datetime.utcnow().strftime("%Y%m%d-%H%M%S.nmea")
    print( "Current datetime as file suffix:                 " + now_str)
    print(f"Most recent GPS data file avail: {gps_data_file}")

else:
    print(f"User specified file: {gps_data_file}")

#### Decode Given `.nmea` File

In [ ]:
import pynmea2
import os

decoded_file_str, _ = os.path.splitext(gps_data_file)
decoded_file_str += ".txt"

# Lookup table for what messages mean (only includes standard NMEA msgs)
# Obtained from NEO-6M datasheet
# https://content.u-blox.com/sites/default/files/products/documents/u-blox6_ReceiverDescrProtSpec_%28GPS.G6-SW-10018%29_Public.pdf
# key = sentence formatter
# value = sentence meaning (human readable)
sentence_type_lookup = {
    "DTM": "Datum Reference",
    "GBS": "GNSS Satellite Fault Detection",
    "GGA": "Global positioning system fix data",
    "GLL": "Latitude and longitude, with time of position fix and status",
    "GPQ": "Poll message",
    "GRS": "GNSS Range Residuals",
    "GSA": "GNSS DOP and Active Satellites",
    "GST": "GNSS Pseudo Range Error Statistics",
    "GSV": "GNSS Satellites in View",
    "RMC": "Recommended Minimum data",
    "THS": "True Heading and Status",
    "TXT": "Text Transmission",
    "VTG": "Course over ground and Ground speed",
    "ZDA": "Time and Date",
}

print(f"Reading file {gps_data_file}\n")

with open(decoded_file_str, "w") as out_file:
    nmea_data = open(gps_data_file, "rb")
    for message_bytes in nmea_data.readlines()[:]: # read all messages
        try:
            message = message_bytes.decode("utf-8").replace("\n", "").replace("\r", "")
            parsed_message = pynmea2.parse(message)
        except:
            # skip invalid messages
            continue
            
        print(f"message: {message}:", file=out_file)
        print(f"message meaning: {sentence_type_lookup[parsed_message.sentence_type]}", file=out_file)

        for field in parsed_message.fields:
            value = getattr(parsed_message, field[1])
            print(f"{field[0]:40} {field[1]:20} {value}", file=out_file)
        
        print("\n", file=out_file)

nmea_data.close()
print(f"Save to {decoded_file_str} completed")

#### Data Extraction from Parsed Messages

In [ ]:
import pynmea2

nmea_data = open(gps_data_file, "rb")

coordinates_data = []

for message_bytes in nmea_data.readlines():    
    try:
        message = message_bytes.decode("utf-8").replace("\n", "").replace("\r", "")
        parsed_message = pynmea2.parse(message)
    except:
        # skip invalid sentences
        continue

    cga_data = {}
    
    # process only GGA messages
    if parsed_message.sentence_type == "GGA":
        for attr in ["timestamp", "latitude", "longitude", "latitude", "horizontal_dil", "num_sats", "gps_qual"]:
            cga_data[attr] = getattr(parsed_message, attr)
        coordinates_data.append(cga_data)
        
coordinates_data[0]

#### Pandas Geo Dataframe

In [ ]:
import pandas as pd
import geopandas as gpd

whole_frame = True

df = pd.DataFrame(coordinates_data)
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude, crs="EPSG:4326"))

if whole_frame:
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.precision', 3):
        display(gdf)
else:
    display(gdf)

#### Render Map of GPS Data

In [ ]:
import matplotlib.pyplot as plt
import contextily as ctx

fig = plt.figure(figsize=(10,10))
ax = plt.axes()
gdf[gdf.gps_qual > 0].plot(ax=ax, alpha=.2, edgecolor="#ffff", color='red')
ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerLite, crs="EPSG:4326", alpha=.3)